In [1]:
import pandas as pd
import gensim
from gensim.test.utils import common_texts
from gensim.models import Word2Vec, KeyedVectors
from gensim import downloader
import gzip 
import math
import itertools
from time import time
from tqdm import tqdm
import tqdm.notebook as tq
import os
os.getcwd()

'G:\\My Drive\\KU\\Thesis'

**Format for ngram documents:**

ngram TAB year TAB match_count TAB page_count TAB volume_count NEWLINE

## Parallel with mgzip

# Partitioning and deleting entries under 40 matches

In [3]:
import mgzip
import string
from datetime import datetime
from time import time

#Differences with this one: it adds up the match count of all the years in a decade and writes one line per decade, 
# threshold is reduced to 25
# gets rid of lines with punctuation

class ParrNgramFiler():
    def __init__(self, coll, out, dire, start_yr, end_yr, matchmin=25, test=False):
        self.coll = coll
        self.out = out
        self.dir = dire
        self.start_yr = start_yr
        self.end_yr = end_yr
        self.decades = [start_yr+i*10 for i in range((end_yr-start_yr)//10)]
        self.matchmin = matchmin
        self.exclist = string.punctuation + string.digits
        self.punc = string.punctuation
        self.lines = {}
    
    def write(self, ngram):
        split = []
        for word in ngram.lower().split():
            word = word.split("_")[0]
            if word in self.punc:
                continue
            else:
                split.append(word)
        if len(split) < 5:
            continue
        else:
            to_write = " ".join(split + ["\t" + str(match_count) + "\n"]) #Get rid of POS tagging on end of words
            if self.test != False:
                 print("TO WRITE: ", to_write)
            split = " ".join(split)
            self.write_dic[current_dec].write(split)
        
    def file(self): 
        os.chdir(f"{self.dir}/raw_data/{self.coll}")
        self.write_dic = {}
        self.size_dic = {}
        self.lines = 0
        if self.test:
            try:
                shutil.rmtree(f"../{self.coll}_{self.out}/")
            except:
                pass
        os.makedirs(f"{self.dir}/processed_data/{self.coll}_{self.out}/")
        for dec in self.decades:
            fname = str(dec) + ".gz"
            self.write_dic[dec] = mgzip.open(f"{self.dir}/processed_data/{self.coll}_{self.out}/" + fname, "wt", encoding="utf-8")
        
        #Try on specified no. files if testing
        if self.test != False:
            file_list = os.listdir()[:test]
        else:
            file_list = os.listdir()
        
        
        for filname in tq.tqdm(file_list):
            
            #Skip over the POS tag files
            if filname[0] = "_":
                continue
            
            print("File: ", filname, f", current time: {time()-start}, current lines: {self.lines}")
            #Open file with parallel gzip 
            with mgzip.open(f"{self.dir}/raw_data/{self.coll}/" + filname, "rt", encoding="utf-8") as fil:
                #Try to counteract "not a gzipped file" error
                try:
                    for i, line in enumerate(tq.tqdm(fil)):
                        
                        #count lines for printing after each file
                        self.lines[fil] += 1
                        
                        #Split file at the tab
                        line = gensim.utils.to_unicode(line).split("\t")
                        
                        #Get the year as integer, throw up an error if no year fuond
                        try:
                            year = int(line[1])
                        except ValueError:
                            print("Year error on line: ", line)
                            continue
                        
                        #Get corresponding decade
                        dec = int(str(year)[:-1] + "0")
                        
                        #Isolate ngram
                        ngram = line[0]
                        
                        #Initialise values for decadising ngram data
                        if i == 0:
                            prev_ngram = ngram
                            match_count = int(line[2])
                            current_dec = dec
                            continue
                            
                        #Get match count
                        try: 
                            matches = int(line[2])
                        except Exception as e:
                            print(f"Error, line {i} -> {line}:\n", e)
                            continue
                        
                        #Print first 1000 if in test mode
                        if self.test != False:
                            if i > self.test:
                                print("\nCurrent ngram: ", prev_ngram, "\nMatches: ", match_count, "\n Current Line: ", line)
                            else:
                                break
                        
                        #check current/previous ngram
                        if prev_ngram == ngram:
                            #check current/previous decade
                            if dec == current_dec:
                                match_count += matches
                            #if not equal, 
                            else: 
                                if match_count > self.matchmin:
                                    split = []
                                    for word in ngram.lower().split():
                                        word = word.split("_")[0]
                                        if word in self.punc:
                                            continue
                                        else:
                                            split.append(word)
                                    if len(split) < 5:
                                        continue
                                    else:
                                        #to_write = " ".join(split + ["\t" + str(match_count) + "\n"]) #Get rid of POS tagging on end of words
                                        #print("TO WRITE: ", to_write)
                                        print(split)
                                        split = " ".join(split)
                                        self.write_dic[current_dec].write(split)
                                        current_dec = dec
                                        match_count = matches
                                else:
                                    current_dec = dec
                                    match_count = matches

                        else:
                            if match_count > self.matchmin:
                                split = []
                                for word in ngram.lower().split():
                                    word = word.split("_")[0]
                                    if word in self.punc:
                                        continue
                                    else:
                                        split.append(word)
                                if len(split) < 5:
                                    continue
                                else:
                                    print(split)
                                    split = " ".join(split)
                                    self.write_dic[current_dec].write(split)
                            prev_ngram = ngram
                            match_count = matches
                            current_dec = dec
                except Exception as e:
                    print(e)




            
                
        for dec in self.decades:
            self.write_dic[dec].close()
                    


start = time()
Filer = ParrNgramFiler(coll="gb_12", out="min_100_test9", dire="D:/google_ngrams", start_yr=1500, end_yr=2010, matchmin=100)
Filer.file(test=5)
print(f"Total time taken: {time()-start}")

SyntaxError: invalid syntax (Temp/ipykernel_10516/1231472528.py, line 65)

## Check whether each file iterated over writes more information to the output files

In [ ]:
NgramFiler.size_dic

